# Sulawesi Fault test
### An example of how to run fakequakes and geoclaw on a new fault

# .utils.py

# fault.py

# Install Cartopy

In [6]:
import cartopy

# Create object

In [7]:
import numpy as np
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
from geopy.distance import geodesic

# Load the SUL slab data
sul_data = np.load('sul_slab.npz')
lat = sul_data['lat']
lon = sul_data['lon']
depth = sul_data['depth']
dip = sul_data['dip']
strike = sul_data['strike']

# Load the HAL slab data
hal_data = np.load('hal_slab.npz')
hal_lat = hal_data['lat']
hal_lon = hal_data['lon']
hal_depth = hal_data['depth']
hal_dip = hal_data['dip']
hal_strike = hal_data['strike']

# Define bounds for the map
bounds = {'lat_max': max(lat[-1], hal_lat[-1]),
          'lat_min': min(lat[0], hal_lat[0]),
          'lon_max': max(lon[-1], hal_lon[-1]),
          'lon_min': min(lon[0], hal_lon[0])}

# Create the GridFault objects
# sul_grid_fault = GridFault(lat, lon, depth, dip, strike, bounds)
# hal_grid_fault = GridFault(hal_lat, hal_lon, hal_depth, hal_dip, hal_strike, bounds)

# Define a grid for latitude and longitude
lon_values = np.linspace(bounds['lon_min'], bounds['lon_max'], 300)
lat_values = np.linspace(bounds['lat_max'], bounds['lat_min'], 300)
Lon, Lat = np.meshgrid(lon_values, lat_values)

# Compute depth maps for the grid
# sul_depth_map = sul_grid_fault.depth_map(Lat, Lon)
# hal_depth_map = hal_grid_fault.depth_map(Lat, Lon)

# # Transpose depth maps
# sul_depth_map = sul_depth_map.T
# hal_depth_map = hal_depth_map.T

# Create a figure
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

# Set map extent
ax.set_extent([bounds['lon_min'], bounds['lon_max'], bounds['lat_min'], bounds['lat_max']], crs=ccrs.PlateCarree())

# Add coastlines
ax.coastlines(resolution='10m')

# Define depth contour levels
depth_levels = np.arange(20, 700, 20) * 1000

# **Plot both faults on the same map**
# SUL slab contours
sul_contour = ax.contour(Lon, Lat, sul_depth_map, levels=depth_levels, cmap='viridis', transform=ccrs.PlateCarree())
sul_fault_contour = ax.contour(Lon, Lat, sul_depth_map, levels=[150000], colors='red', transform=ccrs.PlateCarree())

print(sul_depth_map)

# HAL slab contours
hal_contour = ax.contour(Lon, Lat, hal_depth_map, levels=depth_levels, cmap='viridis', transform=ccrs.PlateCarree())
hal_fault_contour = ax.contour(Lon, Lat, hal_depth_map, levels=[150000], colors='orange', transform=ccrs.PlateCarree())


length_dict = {}


def calculate_fault_midpoint(contour, color, label):
    """
    Calculates the length and midpoint of each individual segment in the contour.
    Plots the midpoints on the map.
    """

    # if label not in length_dict:
    #     length_dict[label] = []  # Store multiple segments per fault label

    segment_index = 1  # Unique identifier for each segment of the fault

    for segment_group in contour.allsegs:  # Each contour level
        # print(f"{segment_group}")
        for segment in segment_group:  # Each individual contour segment
            # print(f"segement: {segment}")
            if len(segment) > 1:
                segment_length = 0
                contour_points = []

                # Compute segment length
                for i in range(len(segment) - 1):
                    point1 = (segment[i][1], segment[i][0])  # lat, lon
                    point2 = (segment[i + 1][1], segment[i + 1][0])
                    small_segment_length = geodesic(point1, point2).meters
                    segment_length += small_segment_length
                    contour_points.append((small_segment_length, point2))

                segment_name = f"{label} Segment {segment_index}"
                print(f"Total fault length for {segment_name}: {segment_length:.2f} meters")


                if segment_length < 20000:
                  break

                length_dict[segment_name] = segment_length  # Store directly as key-value pair


                # length_dict[label].append({segment_name: segment_length})  # Store with unique segment name

                # print(f"Total fault length for {label} segment: {segment_length:.2f} meters")

                # length_dict[label] = segment_length

                # Find midpoint for this segment
                half_length = segment_length / 2
                cumulative_length = 0
                midpoint = None

                for segment_length, point in contour_points:
                    cumulative_length += segment_length
                    if cumulative_length >= half_length:
                        midpoint = point
                        break

                # Plot midpoint if found
                if midpoint:
                    print(f"Midpoint of {label} segment: {midpoint}")
                    ax.plot(
                        midpoint[1], midpoint[0], 'o',
                        color=color, markersize=6,
                        label=f'Midpoint {label}',
                        transform=ccrs.PlateCarree()
                    )

                segment_index += 1

# **Calculate and plot midpoints for both faults**
calculate_fault_midpoint(sul_fault_contour, 'red', 'SUL Fault')
calculate_fault_midpoint(hal_fault_contour, 'orange', 'HAL Fault')

# Add gridlines
gl = ax.gridlines(draw_labels=True)
gl.right_labels = False
gl.top_labels = False

# Add legend and labels
plt.legend()
plt.title('Overlapping Fault Depth Contours (SUL and HAL)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show the plot
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'sul_slab.npz'

## Calculate Lengths and Width functions

In [ ]:
def calc_length(magnitude, delta_logl):
    """Computes the rupture length of the fault based on an earthquake's moment
    magnitude using a regression formula.

    Parameters
    ----------
    magnitude : float
        The moment magnitude of the earthquake event.
    delta_logl : float
        An offset factor for the log of the rupture length.

    Returns
    -------
    length : float
        The rupture length in (meters? km?)
    """
    a = 0.5233956445903871     # slope
    b = 1.0974498706605313     # intercept

    mu_logl = a*magnitude + b
    logl = mu_logl + delta_logl

    return 10**logl

def calc_magnitude(length, delta_logl):
    a = 0.5233956445903871     # slope
    b = 1.0974498706605313     # intercept

    # length_km = length/1000

    # print("length" + type(length))

    # return (np.log(length) - b - delta_logl)/a

    logl = np.log10(length)  # Compute log10 of length
    magnitude = (logl - b - delta_logl) / a  # Solve for magnitude
    return magnitude


def calc_width(magnitude, delta_logw):
    """Computes the rupture width of the fault based on an earthquake's moment
    magnitude using a regression formula.

    Parameters
    ----------
    magnitude : float
        The moment magnitude of the earthquake event.
    delta_logw : float
        An offset factor for the log of the rupture width.

    Returns
    -------
    length : float
        The rupture width in (meters? km?)
    """
    m = 0.29922483873212863   # slope
    c = 2.608734705074858     # y intercept

    mu_logw = m*magnitude + c
    logw = mu_logw + delta_logw
    return 10**logw

In [ ]:
for key, value in length_dict.items():
    print(f"{key}: {value}")

for key, value in length_dict.items():
    # print(type(length_dict[key]))
    print(f"{key}" + " magnitude = " + f"{calc_magnitude(length_dict[key], delta_logl=0)}")
    # print(f"{key}" + " length = " + f"{calc_length(calc_magnitude(length_dict[key], delta_logl=0), delta_logl=0)}")
    print(f"{key}" + " width = " + f"{calc_width(calc_magnitude(length_dict[key], delta_logl=0), delta_logw=0)}")

    #save centroid, length, width, and magnitude as attributes of the GridFault for each fault
    #arrange the code into an organized function and commit it to github
    # make sure that's all we need to give fake quakes
    # look back and look at how the super computer works


SUL Fault Segment 1: 244887.33954676578
HAL Fault Segment 1: 912092.9617962152
HAL Fault Segment 2: 257553.31075518767
SUL Fault Segment 1 magnitude = 8.199373660765755
SUL Fault Segment 1 width = 115396.05617039916
HAL Fault Segment 1 magnitude = 9.290465604911672
HAL Fault Segment 1 width = 244721.8671122068
HAL Fault Segment 2 magnitude = 8.241217348430633
HAL Fault Segment 2 width = 118771.33475629539
